# 法律 RAG Notebook 工作流
此 Notebook 示範如何以模組化方式載入 `rag_system` Workflow，替代舊有 CLI。
1. 載入環境變數並建立 `RAGConfig`。
2. 透過 `rag_system.workflow` 建立 LLM 與 LangGraph Workflow。
3. 以程式方式執行查詢並顯示答案。

In [ ]:
import os
from dotenv import load_dotenv

from rag_system.config import RAGConfig
from rag_system.workflow import create_llm, create_rag_workflow, run_query

load_dotenv()

config = RAGConfig.from_env()
config.top_k = 5  # 視需要調整
config.content_max_length = 800
config.validate()
config

In [ ]:
llm = create_llm(config)
workflow = create_rag_workflow(config, llm=llm)
workflow

In [ ]:
question = "行政程序法第102條規定了什麼？"
result_state = run_query(question, config, llm=llm)
print(result_state.get("generation", "尚未產生答案"))